In [0]:
%run "./02 Creating Spark Data Frame to Select and Rename Columns"

In [0]:
users_df.createOrReplaceTempView('users')

In [0]:
spark.sql("""
    SELECT id, (amount_paid + 25) AS amount_paid
    FROM users
"""). \
    show()

+---+-----------+
 id|amount_paid|
+---+-----------+
 1| 1025.55|
 2| 925.0|
 3| 875.55|
 4| NaN|
 5| NaN|
+---+-----------+

+---+----------+------------+--------------------+--------------------+-------+-----------+-----------+-------------+-------------------+
 id|first_name| last_name| email| phone_numbers|courses|is_customer|amount_paid|customer_from| last_updated_ts|
+---+----------+------------+--------------------+--------------------+-------+-----------+-----------+-------------+-------------------+
 1| Corrie|Van den Oord|cvandenoord0@etsy...|{+1 234 567 8901,...| [1, 2]| true| 1000.55| 2021-01-15|2021-02-10 01:15:00|
 2| Nikolaus| Brewitt|nbrewitt1@dailyma...|{+1 234 567 8923,...| [3]| true| 900.0| 2021-02-14|2021-02-18 03:33:00|
 3| Orelie| Penney|openney2@vistapri...|{+1 714 512 9752,...| [2, 4]| true| 850.55| 2021-01-21|2021-03-15 15:16:55|
 4| Ashby| Maddocks| amaddocks3@home.pl| {null, null}| []| false| NaN| null|2021-04-10 17:45:30|
 5| Kurt| Rome|krome4@shutterfly...|{+1 817 934 7142,...| []| false| NaN| null|2021-04-02 00:55:18|
+---+----------+------------+--------------------+--------------------+-------+-----------+-----------+-------------+-------------------+

In [0]:
users_df. \
    selectExpr('id', '(amount_paid + 25) AS amount_paid'). \
    show()

+---+-----------+
 id|amount_paid|
+---+-----------+
 1| 1025.55|
 2| 925.0|
 3| 875.55|
 4| NaN|
 5| NaN|
+---+-----------+

In [0]:
# This will fail
users_df.select('id', 'amount_paid' + 25).show()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-444060604359967> in <module> 
 1 # This will fail 
 ----> 2 users_df . select ( 'id' , 'amount_paid' + '25' ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1690 [ Row ( name = 'Alice' , age = 12 ) , Row ( name = 'Bob' , age = 15 ) ] 
 1691 """
 -> 1692 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1693 return DataFrame ( jdf , self . sql_ctx ) 
 1694 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : cannot resolve '`amount_paid25`' given input columns: [amount_paid, courses, customer_from, email, first_name, id, is_customer, last_name, last_updated_ts, phone_numbers];
'Project [id#3176L, 'amount_paid25]
+- LogicalRDD [id#3176L, first_name#3177, last_name#3178, email#3179, phone_numbers#3180, courses#3181, is_customer#3182, amount_paid#3183, customer_from#3184, last_updated_ts#3185], false

In [0]:
from pyspark.sql.functions import lit

In [0]:
lit(25)

Out[14]: Column<'25'>

In [0]:
# This will also fail

users_df.select('id', 'amount_paid' + '25').show()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-444060604359977> in <module> 
 1 # This will also fail 
 2 
 ----> 3 users_df . select ( 'id' , 'amount_paid' + '25' ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1690 [ Row ( name = 'Alice' , age = 12 ) , Row ( name = 'Bob' , age = 15 ) ] 
 1691 """
 -> 1692 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1693 return DataFrame ( jdf , self . sql_ctx ) 
 1694 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : cannot resolve '`amount_paid25`' given input columns: [amount_paid, courses, customer_from, email, first_name, id, is_customer, last_name, last_updated_ts, phone_numbers];
'Project [id#3176L, 'amount_paid25]
+- LogicalRDD [id#3176L, first_name#3177, last_name#3178, email#3179, phone_numbers#3180, courses#3181, is_customer#3182, amount_paid#3183, customer_from#3184, last_updated_ts#3185], false

In [0]:
from pyspark.sql.functions import lit, col

In [0]:
'amount_paid' + lit(25.0)

Out[17]: Column<'(25.0 + amount_paid)'>

In [0]:
'amount_paid' + lit('25.0')

Out[18]: Column<'(25.0 + amount_paid)'>

In [0]:
# Returns null
# amount_paid is converted to string by implicitly using lit.
# Spark returns null when we perform arithmetic operations on noncompatible types
users_df.select('id', 'amount_paid' + lit(25.0), lit(50) + lit(25)).show()

+---+--------------------+---------+
 id|(25.0 + amount_paid)|(50 + 25)|
+---+--------------------+---------+
 1| null| 75|
 2| null| 75|
 3| null| 75|
 4| null| 75|
 5| null| 75|
+---+--------------------+---------+

In [0]:
# This works
users_df.select('id', col('amount_paid') + lit(25.0)).show()

+---+--------------------+
 id|(amount_paid + 25.0)|
+---+--------------------+
 1| 1025.55|
 2| 925.0|
 3| 875.55|
 4| NaN|
 5| NaN|
+---+--------------------+

In [0]:
# lit returns column type
lit(25.0)

Out[24]: Column<'25.0'>